# Exercise: Reproducing 'How close is "close"?' (Shingleton & Basiri, 2025)

This notebook guides you through a **small-scale reproduction** of the core analytical pipeline in:

- Shingleton, J., & Basiri, A. (2025). How close is "close"? An analysis of the spatial characteristics of perceived proximity using Large Language Models. AGILE GIScience Series, 6, 1–14. [10.1111/1755-2665.12276](https://doi.org/10.1111/1755-2665.12276)

## 🎯 Objectives of the Notebook

1. Load **Inside Airbnb** listings for a city (London in the paper).
2. Use a **Large Language Model (LLM)** to extract *explicitly named* places described as **near** each property.
3. Robustly parse and clean the model output (incl. **JSON repair** and **fuzzy hallucination filtering**).
4. Compute and interpret a **standard distance** for frequently mentioned (“reference”) locations.

---

## Working assumptions (for a classroom reproduction)

- We run the LLM **locally** (via Ollama) and process a **small sample** (e.g. 200–500 listings).
- We reproduce the *core logic* of the paper, not the full London-scale compute budget.

> Keep notes as you go: any simplification you make is part of your reproducibility argument.


## 0. Environment setup

This exercise uses common data-science and geospatial libraries. If your environment is missing packages, install them below.

**Libraries you will use**
- `pandas` for tabular data
- `geopandas` + `shapely` for geospatial data structures
- `h3` for hexagonal indexing

- `tqdm` for progress bars, because we are processing a lot of data

- `ollama` for local LLM calls (or skip if you use another provider)
- `json_repair` to handle malformed JSON from the LLM
- `rapidfuzz` for fuzzy matching to remove hallucinated place names


In [ ]:
# If needed, uncomment and run:
# %pip install -q pandas geopandas shapely h3 tqdm matplotlib ollama json_repair rapidfuzz

## 1. Imports

Read this cell carefully: in a reproducible notebook, you want *all imports in one place* so dependencies are obvious.

In [ ]:
import json
import math
import re

import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

from tqdm.auto import tqdm

import h3
import ollama
from rapidfuzz import fuzz
from json_repair import repair_json

# progress bar initialization
tqdm.pandas()

## 2. Data: Inside Airbnb listings

The paper uses Inside Airbnb listings for **London** and analyses property descriptions + coordinates. We will do the same analysis for Austria.

For this exercise:
- Open the [Inside Airbnb](https://insideairbnb.com/) website
- Find a city in Austria. 
```python
❓ `Is Graz Available?`
```
- Download `listings.csv.gz` from Inside Airbnb, then set `DATA_PATH` below.

**Task (fill-in):** set `DATA_PATH` to your local file.


In [ ]:
# TODO (fill in): path to your downloaded listings.csv or listings.csv.gz
DATA_PATH = ___


## 3. Load and inspect the listings

We will keep only the columns we need:
- `id` (listing identifier)
- `latitude`, `longitude`
- `description` (the key text input to the LLM)
- `neighborhood_overview` (for additional location context)
- `neighbourhood_cleansed` (for later filtering)

**🚀 Tasks**
1. How many total listings is present in the dataset?
2. What does a typical description/neighborhood_overview look like?
3. How many listings have missing descriptions and neighborhood_overview ? 
4. Create a kepler.gl map of all listings density based on h3 cells
```python
❓ `What is the coordinate reference system (CRS) of the data? Is it acceptable for our analysis?`
```

In [ ]:
# load dataframe 
raw_airbnb_df = pd.___(DATA_PATH)

# TODO: print the number of rows of the dataframe


In [ ]:
# Keep only relevant columns
use_cols = [
    ___
]

airbnb_df = raw_airbnb_df[___].copy()


In [ ]:
# Clean na values
# drop any row that has missing values in the columns we need
airbnb_df = airbnb_df.dropna(subset=[___])

# You can also use the parameter how='all' to drop only rows that have missing values in all columns
# In the next step we will concatenate "description" and "neighborhood_overview"
airbnb_df = airbnb_df.dropna(subset=[___], how='all')

# TODO: print the number of of row in the cleaned sample of the dataframe


In [ ]:
# TODO: what look like the data? especially the description column

## 4. Spatial aggregation with H3

The paper aggregates properties into an H3 grid at **resolution 7** (~5.16 km² per hexagon in their setup).  
We will use the same default so your maps are comparable in scale.

**🚀 Tasks** 
1. assign for each row a h3 cell id using the `point_to_h3` function
2. group by h3 cell id and count the number of properties in each cell
3. create a kepler.gl map of the h3 cell density


In [ ]:
# TODO: create a function that will be `apply` to your dataframe 

H3_RES = 7

def latlon_to_h3(___) -> str:
    """Return the H3 cell id for a (lat, lon) coordinate."""
    return ___

airbnb_df["h3_cell"] = airbnb_df.apply(point_to_h3, axis=1)

In [ ]:
# TODO: group your data by h3 cell and count the number of properties in each cell
h3_airbnb_density = (
    airbnb_df
    .groupby("___")
    .___
    .reset_index(name="airbnb_count")
)

In [ ]:
# TODO: create a kepler.gl map of the h3 cell density

## 5. Filter your data to only include the inner city

We have definitely too many points in our dataset to do the analysis. Let's filter to only include the inner city.

**🚀 Task:**
1. Filter your data to only include the inner city
2. Create a kepler.gl map of the inner city


In [ ]:
# TODO: keep only data points from the inner city
# tip: use the `neighbourhood_cleansed` column, what are the possible values? 

# TODO: How many data points are left?


In [ ]:
# TODO: create a kepler.gl map of the inner city


## 6. Definition of the extraction prompt

The paper uses a carefully engineered prompt (Appendix Fig. A1) to extract **explicitly named** nearby locations, excluding vague places and excluding the property’s own location.

**🚀 Tasks**
1. Read the following prompt for Vienna, is it similar of the one you are used to write?
2. Identify **three** design choices that reduce false positives
3. What are the differences between this prompt and the one used in the paper?


In [ ]:
SYSTEM_PROMPT = """
You are an Airbnb location extraction system for Vienna. 
Your task is to extract specific Points of Interest (POIs) that are mentioned as being NEAR the property.

### 1. WHAT TO EXTRACT (Positive Criteria)
You must only extract specific, physical landmarks that can be pinpointed on a map.
- Specific Landmarks (e.g., "Schönbrunn Palace", "St. Stephen's Cathedral")
- Named Squares/Streets (e.g., "Stephansplatz", "Mariahilfer Straße", "Florianigasse")
- Named Markets/Parks (e.g., "Naschmarkt", "Prater", "Schönbornpark")
- Specific Transport Hubs (e.g., "Westbahnhof", "U4 Pilgramgasse")

### 2. WHAT TO IGNORE (Negative Criteria)
- IGNORE Districts and Quarters: Do NOT extract "Neubau", "Favoriten", "1st District", "Freihausviertel", "Museumsquartier" (unless referring to the specific complex, usually assume it's a district).
- IGNORE Generic Amenities: Do NOT extract "supermarkets", "restaurants", "bars", "shops", "metro station" (if unnamed).
- IGNORE Property Location: If the text says "The apartment is located in [Place]," ignore [Place]. Only extract places described as being *near*, *around*, or *walking distance* from the apartment.

### 3. ADDRESS FORMATTING RULES
- If a full address is in the text, use it.
- If NO address is in the text, you MUST format the address as: "[Name], Vienna".
- This is required formatting, not "inventing."

### 4. OUTPUT FORMAT
Do not include markdown formatting or explanations.
Keys: "name", "address", "latitude" (default null), "longitude" (default null), "context".

### 5. FEW-SHOT EXAMPLES (Do not confuse these with your real task)

Input: "We are located in the heart of Neubau, close to the Zieglergasse station and a short walk to Mariahilfer Straße."
Output:
[
  {
    "name": "Zieglergasse station",
    "address": "Zieglergasse station, Vienna",
    "latitude": null,
    "longitude": null,
    "context": "close to the Zieglergasse station"
  },
  {
    "name": "Mariahilfer Straße",
    "address": "Mariahilfer Straße, Vienna",
    "latitude": null,
    "longitude": null,
    "context": "short walk to Mariahilfer Straße"
  }
]

Input: "Enjoy the many shops and cafes in the 7th district. The bus takes you to the City Center."
Output:
[]

Input: "The apartment is located in the 3rd district, right next to the Belvedere Palace and just a short walk from the Rennweg station."
Output:
[
  {
    "name": "Belvedere Palace",
    "address": "Belvedere Palace, Vienna",
    "latitude": null,
    "longitude": null,
    "context": "right next to the Belvedere Palace"
  },
  {
    "name": "Rennweg station",
    "address": "Rennweg station, Vienna",
    "latitude": null,
    "longitude": null,
    "context": "short walk from the Rennweg station"
  }
]
Input: "You are staying directly opposite the famous Hundertwasserhaus."
Output:
[
  {
    "name": "Hundertwasserhaus",
    "address": "Hundertwasserhaus, Vienna",
    "latitude": null,
    "longitude": null,
    "context": "directly opposite the famous Hundertwasserhaus"
  }
]

### END OF EXAMPLES
Now, analyze the following description and extract the locations based strictly on the text provided below.
"""

## 7. Calling an LLM locally (Ollama)

We will call a local model via Ollama.
- You need [Ollama](https://ollama.com/) running locally. Follow the link to download and install it.
- You also need a model pulled (e.g., `llama3`, `qwen2`, etc.). You can find a list of available models [here](https://ollama.com/models).

Running LLMs locally is extremely expensive for your RAM, Hard Drive and CPU. As a rule of thumb you need 1GB of RAM and 1GB of Hard Drive per 1B parameters. We will use a small model for this exercise: `qwen3:1.7b`.

**🚀 Task** 
1. set `MODEL_NAME` to a model you have locally
2. Pull the model using `ollama.pull`


In [ ]:
MODEL_NAME = "qwen3:1.7b" # You need 1.4GB of free space on your computer
# ⚠️ You must have Ollama running locally 

ollama.pull(MODEL_NAME)

# Create a client to call your model 
client = ollama.Client()


### 7.1 A minimal chat wrapper

- We keep temperature = 0 for deterministic outputs (useful in reproduction).
- We allow the model to think (set `think=True`), which is useful for debugging. Not all models support this.

In [ ]:
def ollama_extract_nearby(text: str, show_log: bool = True) -> str:
    """Return the raw model text output for one listing."""

    # define what you send to the model, the `system`` defines how the model should react, the `user` is the text you want to analyze 
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": text},
    ]
    # you call the model here 
    resp = client.chat(
        model=MODEL_NAME,
        messages=messages,
        options={"temperature": 0}, 
        think=True
    )

    if show_log:
        # print the model's thinking process
        print("-----")
        print(description)
        print("\n thinking... \n")
        print("response",response["message"].get("thinking", "...no thinking..."))
        print("\n\n =>\n")
        print(content)
    
    # return only the content of the response without the thinking part 
    return resp["message"]["content"]


### 7.2 Try the model on one listing

**🚀 Task:** 
1. Run the model on one description of your dataframe 
```python
❓ `What is the thinking process of the model looking like?`
❓ `Are the results correct?`
❓ `How long does it take to run?`
❓ `What happens if your rerun the same description? Is the output the same?`
❓ `What happens if you change thinking to False?`
```

In [ ]:
# TODO: Run your function on the first row of the dataframe



## 8) Parsing the LLM output robustly

In practice, models sometimes return malformed JSON. The paper repairs JSON first, and may ask the model to fix it if needed.

We will implement a two-step parser:
1. Try `json.loads`
2. If that fails, try `json_repair.repair_json`, then parse again

**🚀 Tasks** 
1. creaze the function `parse_json_array` that clean the output of the LLM and try to parse it as a JSON array


In [ ]:
def parse_json_array(text: str) -> List[Dict[str, Any]]:
    """Parse a JSON array from model output, with basic repair."""
    text = text.strip()

    # 1) direct parse
    # TODO: wrap the json load in a try-except block
    data = json.loads(text)
    if isinstance(data, list):
        return data

    # 2) attempt repair
    # TODO: use the library `json_repair` to repair the json
    repaired = ___  # returns a JSON string
    
    # TODO: try to reload the repaired json

    # 3) final fallback: return empty
    return []


In [ ]:
# TODO: Use the output of your LLM from earlier to parse the JSON array

## 9) Batch extraction on a small sample

We will:
- run the LLM on each airbnb listing
- parse the JSON
- store results in a new column `nearby_raw`

**🚀 Task** 
1. ⚠️ Let select a very small sample (e.g. 3-10 rows) to test and debug your code
2. Create a function that will be apply to each row of the dataframe to extract the locations mentioned in the description. The function should run your `ollama_extract_nearby` function and parse the JSON
3. Apply the function to the dataframe and store the results in a new column `nearby_raw`
4. Compute the mean number of extracted locations per listing
5. Create a Kepler.gl map to visualize the extracted locations

**📝 Note**
- The execution might be long... so we will use `progress_apply` from `tqdm` to show a progress bar (instead of a simple `apply`)

> **📢 TODO at home:**  
> Run the operation on the entire dataframe and save the results

In [ ]:
# get a small sample of the dataframe
small_sample = airbnb_df.sample(___)

In [ ]:
# create a function that will be apply on your df
def extract_for_row(row: str) -> List[Dict[str, Any]]:

    # TODO: aggregate the columns

    # you can also clean the text
    ___ = ___.replace("\n", " ") # remove new lines
    ___ = ___.replace("<br />", " ") # remove html tags
    ___ = ___.trim().lower() # remove extra spaces and lower case

    if len(___)==0: # make sure the text is not empty 
        return []
    
    # TODO: run your LLM function

    # TODO: Return the parsed value


In [ ]:
airbnb_df["nearby_raw"] = ___.progress_apply(___)

In [ ]:
# Let's save our results here if you ran all your dataframe 

# In json
JSON_FILEPATH = ___
airbnb_df.to_json(JSON_FILEPATH)

# in parquet
PARQUET_FILEPATH = ___
airbnb_df.to_parquet(PARQUET_FILEPATH)

```python
? `What is the difference between parquet and json? check on your disk the space of each`
```

In [ ]:
# TODO: calculate the number of locations per row and add it as a new column
airbnb_df["nearby_raw_n"] = ___


In [ ]:
# TODO: create a kepler map that show the average number of nearby locations per cell

In [ ]:
# adding more soon...

**Congratulations! 🎉 You have successfully completed the exercise.**